In [1]:
import os
os.chdir("C:\\Users\\erin.melly\\Desktop\\GitHub\\Tax-Calculator_dist_table")
from behresp import *
from taxcalc import *
import pandas as pd
import numpy as np

In [2]:
# dictionary of assumed elasticity each year
cg= {2021: -4.184, 2022: -3.488, 2023: -2.76, 
     2024: -2.76, 2025: -2.76, 2026: -2.76, 
     2027: -2.76, 2028: -2.76, 2029: -2.76, 
     2030: -2.76
    }

In [11]:
def create_biden_distribution_table(cyr, taxref, group):
    assert taxref in ['individual', 'individual_business']
    assert group in ['standard_income_bins', 'weighted_deciles']
    assert cyr in range(2021, 2031)
    if taxref=='individual':
        # baseline
        rec = Records()
        pol = Policy()
        calc_base=Calculator(pol, rec)
        calc_base.advance_to_year(cyr)
        calc_base.calc_all()
        #reform
        rec = Records()
        pol = Policy()
        pol.implement_reform(Policy.read_json_reform('taxcalc\\reforms\\Biden.json'))
        calc_ref= Calculator(pol, rec)
        calc_ref.advance_to_year(cyr)
        calc_ref.calc_all()
    else:
        # baseline
        rec = Records()
        pol = Policy()
        baseline={
            "Business_Effect_Table_expinc": {cyr: True}
        }
        pol.implement_reform(baseline)
        calc_base= Calculator(pol, rec)
        calc_base.advance_to_year(cyr)
        calc_base.calc_all()
        # reform
        rec = Records()
        pol = Policy()
        pol.implement_reform(Policy.read_json_reform('taxcalc\\reforms\\Biden_distributional.json'))
        calc_ref= Calculator(pol, rec)
        calc_ref.advance_to_year(cyr)
        calc_ref.calc_all()   
    # implement behavioral responses
    response_elasticities = {'sub': 0.25, 'cg': cg[cyr]}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities, chg_inc=True)
    # create difference table (1)
    vdf1 = base_df[DIFF_VARIABLES]
    vdf2 = ref_df[DIFF_VARIABLES]
    distdf1 = create_difference_table(vdf1, vdf2, groupby=group, tax_to_diff='combined', pop_quantiles=False)
    # create difference table (2) [only use the pc_aftertaxinc column from this distribution table]
    response_elasticities = {'sub': 0.25, 'cg': cg[cyr]}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities, chg_inc=False)
    vdf1 = base_df[DIFF_VARIABLES]
    vdf2 = ref_df[DIFF_VARIABLES]
    distdf2 = create_difference_table(vdf1, vdf2, groupby=group, tax_to_diff='combined', pop_quantiles=False)
    # append pc_aftertaxinc from dataframe 2 and delete unused columns
    df = distdf1.assign(pch_chg_inc = distdf2['pc_aftertaxinc'])
    table = df.drop(['ubi', 'benefit_cost_total', 'benefit_value_total', 'pc_aftertaxinc'], axis=1)
    # final table, by standard income bin
    return table


#### INDIVIDUAL REFORMS

In [7]:
create_biden_distribution_table(cyr=2021, taxref='individual', group='standard_income_bins')

#### INDIVIDUAL AND BUSINESS REFORMS

In [12]:
create_biden_distribution_table(cyr=2021, taxref='individual_business', group='standard_income_bins')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,pch_chg_inc
<$0K,1.32,0.37,28.4,0.94,71.5,7007.1,9.240,3.1,5.5
=$0K,3.51,0.00,0.0,0.10,3.0,12.2,0.043,0.0,-9.0
$0-10K,19.80,0.36,1.8,15.46,78.1,20.9,0.414,0.1,-0.5
$10-20K,22.86,0.61,2.7,15.56,68.1,39.1,0.895,0.3,-0.3
$20-30K,18.84,0.89,4.7,14.35,76.2,67.5,1.272,0.4,-0.3
$30-40K,16.47,0.82,5.0,13.96,84.8,105.7,1.742,0.6,-0.3
$40-50K,12.96,0.71,5.5,11.20,86.5,153.8,1.993,0.7,-0.4
$50-75K,25.23,1.27,5.1,22.72,90.0,266.3,6.721,2.3,-0.5
$75-100K,15.91,0.89,5.6,14.64,92.1,404.4,6.432,2.2,-0.6
$100-200K,28.07,1.70,6.0,26.08,92.9,713.8,20.033,6.8,-0.6
